In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves import cPickle as pickle
import skimage
import skimage.io
import skimage.transform
import vgg16
import tensorflow as tf
from six.moves import range
import math
# Config the matlotlib backend as plotting inline in IPython
#%matplotlib inline

In [ ]:
images = tf.placeholder("float", [None, 224, 224, 3])
vgg = vgg16.Vgg16()
with tf.name_scope("content_vgg"):
    vgg.build(images)
def vgg_result(batch):
    n=batch.shape[0]
    prob=np.ndarray((0, 1000), dtype=np.float32)
    i=0   
    with tf.Session(
            config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)))) as sess:
        while  i<n :
            j=min(i+100,n)
            print ('1')
            if j<n:
              feed_dict = {images: batch[i:j,:,:,:]}
            else: 
              feed_dict={images:batch[i:,:,:,:]}
           
            prob1 = sess.run(vgg.prob, feed_dict=feed_dict)
            prob=np.concatenate((prob,prob1),axis=0)
            i=j
    return prob

In [ ]:
image_size = 224 # Pixel width and height.
train_folders = [
    os.path.join('train_data', d) for d in sorted(os.listdir('train_data')) if os.path.isdir(os.path.join('train_data',d))]

'''test_folders=[
    os.path.join('test_data', d) for d in sorted(os.listdir('test_data'))]'''

def load_image(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size,3),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      img=  skimage.io.imread(image_file)
      img=img/255.0
      assert  (0 <= img).all() and (img <= 1.0).all()
       
      if len(img.shape)==2:
        img.resize(img.shape[0],img.shape[1],1)
        img=np.repeat(img,3,2)
      elif img.shape[2]>3:
        img=img[:,:,0:3]
      short_edge = min(img.shape[:2])
      yy = int((img.shape[0] - short_edge) / 2)
      xx = int((img.shape[1] - short_edge) / 2)
      crop_img = img[yy: yy + short_edge, xx: xx + short_edge,:]
    # resize to 224, 224
      resized_img = skimage.transform.resize(crop_img, (224, 224))
      image_data=resized_img 
      #if image_data.shape!=(image_size,image_size,3):
         #break
      dataset[num_images, :, :,:] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
   # except ValueError as e:
    #  print ('Could not load')
    
  dataset = dataset[0:num_images, :, :,:]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)

  return vgg_result(dataset)
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_image(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 1)
#test_datasets = maybe_pickle(test_folders, 1)

In [ ]:
def make_arrays(percent):
  if percent>0.01:
    dataset = np.ndarray((0, 1000), dtype=np.float32)
    labels = np.ndarray((0,), dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, valid_percent,test_percent):
  num_classes = len(pickle_files)
  train_dataset,train_labels=make_arrays(1-valid_percent-test_percent)
  valid_dataset,valid_labels=make_arrays(valid_percent)
  test_dataset,test_labels=make_arrays(test_percent)
 
  for label, pickle_file in enumerate(pickle_files):       
    try:
      print(label)
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
       # np.random.shuffle(letter_set)
        print(label)
        temp=-1
        if valid_dataset is not None:
          temp=int(letter_set.shape[0]*valid_percent)
          temp1=letter_set[0:temp, :]
          temp2=np.ones([temp,])*label
          valid_dataset=np.concatenate((valid_dataset,temp1),axis=0)
          valid_labels=np.concatenate((valid_labels,temp2),axis=0)
        if test_dataset is not None:
          temp_1=int(letter_set.shape[0]*test_percent)
          temp1_1=letter_set[temp:temp_1+temp, :]
          temp2_1=np.ones([temp_1,])*label
          test_dataset=np.concatenate((test_dataset,temp1_1),axis=0)
          test_labels=np.concatenate((test_labels,temp2_1),axis=0)
        temp3= letter_set[temp+temp_1:, :]
        temp4=np.ones([temp3.shape[0],])*label
        train_dataset=np.concatenate((train_dataset,temp3),axis=0)
        train_labels=np.concatenate((train_labels,temp4),axis=0)
        print (train_dataset.shape)
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return test_dataset,test_labels,valid_dataset, valid_labels, train_dataset, train_labels
            
            


test_dataset,test_labels,valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, 0.1,0.1)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [ ]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

In [ ]:

        
pickle_file = 'indoor_outdoor.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
     'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise


In [ ]:
global num_labels
image_size = 224
num_labels = max(train_labels)+1

def reformat(labels):

  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return labels
train_labels = reformat(train_labels)
valid_labels = reformat(valid_labels)
test_labels = reformat(test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
pickle_file = 'indoor_outdoor_1.pickle'
try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [ ]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)